# 모듈 프로젝트 1

1. 훈련된 모델을 로드
2. 안면 이미지 파일 혹은 캠을 통한 안면 인식
3. 인식된 안면의 특징 좌표값과 데이터베이스에 저장된 사용자 좌표값을 비교
4. 좌표값 비교(벡터 거리 연산 혹은 다른 비교 연산)하여 비교된 값이 임계값보다 작을 경우 &rarr; 허용된 사용자, 클 경우 &rarr; 침입자

In [ ]:
!pip install streamlit

In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
from PIL import Image, UnidentifiedImageError
import numpy as np

# 모델 로드
def load_model():
    try:
        st.success('모델을 로드 했습니다.', icon="ℹ️")
        return tf.keras.models.load_model('cat_dog_classifier.keras')
    except:
        st.error('모델을 로드할 수 없습니다. 경로를 확인해주세요.', icon="🚨")
        #raise e

model = load_model()

# 업로드된 이미지 전처리
def preprocess_image(image):
    try:
        image = image.resize((150, 150)) # 이미지 크기 조정
        image = np.array(image) / 255.0 # 정규화
        if image.shape[-1] != 3:
            raise ValueError('이미지는 RGB 형식의 컬러 이미지여야 합니다.')
        image = np.expand_dims(image, axis=0) # (1, 150, 150, 3)
        return image
    except Exception as e :
        st.error(f'이미지 전처리중 오류 발생 : {e}', icon="🚨")
        return None

st.title("직원 안면 인식")
st.write("직원의 얼굴을 인식하여 접근 제어를 실시합니다.")

name = st.text_input("이름을 입력하세요:")
password = st.text_input("비밀번호를 입력하세요:", type="password")
if st.button("제출"):
    if password != "1234":
        st.warning("비밀번호가 틀렸습니다!", icon="⚠️")
    else:
        st.write(f"안녕하세요, {name}님!")

upload_file = st.file_uploader('이미지를 업로드하세요!',type=['jpg', 'png', 'jpeg'])
if upload_file is not None:
    try:
        # 이미지 로드
        # 업로드 된 파일을 이미지 객체로 변환
        image = Image.open(upload_file)

        st.image(image, caption='업로드 한 이미지', use_container_width=True)

        preprocessed_image = preprocess_image(image) # 이미지 전처리
        if preprocessed_image is not None:
            prediction = model.predict(preprocessed_image)
            print(prediction)

            # 결과 표시
            if prediction[0][0] > 0.5 :
                st.success('이 이미지는 개로 분류되었습니다.')
            else:
                st.success('이 이미지는 고양이로 분류되었습니다.')
    except UnidentifiedImageError: # 이미지 형식이 아닌 경우 발생
        st.error('이미지를 로드할 수 없습니다. 지원되지 않는 파일 형식입니다.', icon="🚨")
    except Exception as e :
        st.error(f'예측 처리 중 오류 발생 : {e}', icon="🚨")

st.info('note : 이 모델은 학습된 데이터에 따라 결과가 달라질 수 있습니다.')

In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok

In [ ]:
ngrok.set_auth_token('2rF65kRJolLRhMI066tmolBpo1w_4Xdvg9jMHdcJb4a6aqv9A')

In [ ]:
!streamlit run app.py --server.port 8501 &>/dev/null&

In [ ]:
public_url = ngrok.connect(addr="8501")
print(public_url)

In [ ]:
!ps